# Basic implementation

Author: Frankie Inguanez <br />
Date: 13/01/2023<br /><br />

A base implementation of a 9x9 sudoku puzzle, that does a raster scan (left to right, top to bottom) to search for an unknown cell, with an ordered guess. This algorithm is a backtracking (brute-force) algorithm.

In [6]:
import sudokuPuzzleUtils as spu

In [2]:
# Finds the next empty cell in a raster fashion.
def findRaster(puzzle):
    for row in range(len(puzzle)):
        for col in range(len(puzzle[0])):
            if puzzle[row][col] == 0:
                return (row, col)
            
    return None

In [9]:
# Solves a sudoku 9x9 puzzle in a raster pattern using sequential brute force guessing.
def solveRaster(puzzle):
    find = findRaster(puzzle)

    if not find:
        return True
    else:
        row, col = find

    for i in range(1,10):
        if spu.isValid(puzzle, i, (row, col)):
            puzzle[row][col] = i

            if solveRaster(puzzle):
                return True

            puzzle[row][col] = 0

    return False

In [10]:
puzzle = spu.to2DArray('100008509000000000460000000600000000085020047094600302000003090920000700000107005')

print("Solved puzzle!" if solveRaster(puzzle) else "Could not solve puzzle")
puzzle

Solved puzzle!


[[1, 3, 7, 4, 6, 8, 5, 2, 9],
 [2, 5, 8, 3, 1, 9, 4, 7, 6],
 [4, 6, 9, 5, 7, 2, 1, 8, 3],
 [6, 1, 2, 7, 3, 4, 9, 5, 8],
 [3, 8, 5, 9, 2, 1, 6, 4, 7],
 [7, 9, 4, 6, 8, 5, 3, 1, 2],
 [5, 7, 6, 2, 4, 3, 8, 9, 1],
 [9, 2, 1, 8, 5, 6, 7, 3, 4],
 [8, 4, 3, 1, 9, 7, 2, 6, 5]]

In [13]:
def solvePuzzles(puzzlesFileName, statsFileName, errorsFileName, limit):
    import tqdm

    if not limit:
        limit = spu.getFileLineCount(puzzlesFileName)

    i = 0
    hasError = False
    try:
        print("Starting sudoku base solver.")
        
        with open(statsFileName, "w", encoding="utf-8") as sf:
            with open(puzzlesFileName, "r", encoding="utf-8") as pf:
                for line in tqdm.tqdm(pf, total=limit):
                    # Parse the content
                    data = line.split(',')
                    p = data[1].strip()

                    puzzle = spu.to2DArray(p)

                    if not solveRaster(puzzle):
                        sf.write("Could not solve puzzle {:0.0f}".format(data[0]))

                    i+=1

                    if (i>limit):
                        break

    except Exception as e:
        hasError = True
        spu.saveError(e, errorsFileName)

    print("Operation encountered some errors. Check {} for details or script output above.".format(errorsFileName) \
        if hasError else "Sudoku puzzles solved completed successfully.")
    print("Sudoku solver statistics saved in {}".format(statsFileName))


In [14]:
solvePuzzles("sudokuDataset.csv","solverStats.txt", "solverErrors.txt",100)

Starting sudoku base solver.


100%|██████████| 100/100 [00:00<00:00, 1074.93it/s]

Sudoku puzzles solved completed successfully.
Sudoku solver statistics saved in solverStats.txt
